In [ ]:
#| default_exp data

In [ ]:
#| export
import numpy as np
import tensorflow as tf

In [ ]:
#| hide
tf.config.set_visible_devices([], device_type='GPU')

In [ ]:
#| exporti
def load_cifar10():
    from tensorflow.keras.datasets import cifar10
    
    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
    X_train = X_train/255.
    X_test = X_test/255.
    Y_train = Y_train.astype(np.int32)[...,0]
    Y_test = Y_test.astype(np.int32)[...,0]
    
    return tf.data.Dataset.from_tensor_slices((X_train, Y_train)), tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
#| exporti
def load_mnist():
    from tensorflow.keras.datasets import mnist
    
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    X_train = X_train[...,None]/255.
    X_test = X_test[...,None]/255.
    Y_train = Y_train.astype(np.int32)
    Y_test = Y_test.astype(np.int32)

    return tf.data.Dataset.from_tensor_slices((X_train, Y_train)), tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
#| exporti
def load_cats_vs_dogs(test_split):
    import tensorflow_datasets as tfds
    def preprocess(img):
        img = tf.cast(img, tf.float32) / 255.0
        return tf.image.resize(img, size=(150,150))
    dst_train = tfds.load("cats_vs_dogs", split=f"train[:{int(100-test_split*100)}%]")
    dst_val = tfds.load("cats_vs_dogs", split=f"train[{int(100-test_split*100)}%:]")
    dst_train = dst_train.map(lambda x: (preprocess(x["image"]), x["label"]), num_parallel_calls=tf.data.AUTOTUNE)
    dst_val = dst_val.map(lambda x: (preprocess(x["image"]), x["label"]), num_parallel_calls=tf.data.AUTOTUNE)
    return dst_train, dst_val

In [ ]:
#| export
def load_data(name: str,
              test_split: float):
    if name == "cifar10":
        dst_train, dst_val = load_cifar10()

    elif name == "mnist":
        dst_train, dst_val = load_mnist()

    elif name == "cats_vs_dogs":
        dst_train, dst_val = load_cats_vs_dogs(test_split=test_split)
    
    return dst_train, dst_val

In [ ]:
for name in ["mnist", "cifar10", "cats_vs_dogs"]:
    dst_train, dst_val = load_data(name, test_split=0.2)
    print(f"{name}: {len(dst_train)} & {len(dst_val)}")

mnist: 60000 & 10000
cifar10: 50000 & 10000
cats_vs_dogs: 18610 & 4652
